# ДЗ 2. CNN, FFN
### Таня Казакова

In [2]:
pip install torchvision 

In [3]:
!pip install torchmetrics

     |████████████████████████████████| 332 kB 5.3 MB/s 


In [4]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 1.7 MB/s 


In [5]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import collections
from collections import Counter
import torch
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torchmetrics import F1
from torchmetrics.functional import f1, recall
from tqdm import tqdm
tqdm.pandas()
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def lemmatizing(text):
    toks = word_tokenize(text)
    words = [w.lower() for w in toks if w.isalpha()]
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return lemmas

In [7]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Это бывшая колабтетрадка, если захотите перезапускать, исправьте ссылки на твиты (positive.csv и negative.csv)

In [10]:
colnames=['id', 'date', 'name', 'text', 'type', 'rep', 'rtw', 'fav', 'stcount', 'foll', 'frien', 'listcount']
posit = pd.read_csv('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/positive.csv', sep=";", names=colnames)
negat = pd.read_csv('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/negative.csv', sep=";", names=colnames)

In [11]:
posit['sent'] = 1
negat['sent'] = 0
twits = pd.concat([posit,negat],axis=0, ignore_index=True)
twits

,id,date,name,text,type,rep,rtw,fav,stcount,foll,frien,listcount,sent
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,1
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,1
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,1
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,1
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0,0


In [12]:
twits['words'] = twits['text'].progress_apply(lambda x: lemmatizing(x))
twits

100%|██████████| 226834/226834 [12:21<00:00, 305.76it/s]


,id,date,name,text,type,rep,rtw,fav,stcount,foll,frien,listcount,sent,words
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,1,"[хоть, я, и, школотый, но, поверь, у, мы, то, ..."
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,1,"[да, он, немного, похожий, на, он, но, мой, ма..."
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,1,"[rt, katiacheh, ну, ты, идиотка, я, испугаться..."
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,1,"[rt, кто, то, в, угол, сидеть, и, погибать, от..."
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,1,"[вот, что, значит, страшилка, d, но, блин, пос..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0,0,"[но, не, каждый, хотеть, что, то, исправлять, ..."
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0,0,"[скучать, так, только, taaannyaaa, вправлять, ..."
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0,0,"[вот, и, в, школа, в, говно, это, идти, уже, н..."
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0,0,"[rt, lisaberoud, тауриэль, не, грусть]"


In [14]:
# Делаем word2id и id2word

vocab = Counter()
for twit in twits['words']:
    for word in twit:
        vocab[word] +=1
print('Всего уникальных символов:', len(vocab))
filtered_vocab = set()
for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('Уникальных символов, втретившихся больше 2 раз:', len(filtered_vocab))
#создаем словарь с индексами symbol2id, для спецсимвола паддинга дефолтный индекс - 0
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)
#обратный словарь для того, чтобы раскодировать последовательность
id2word = {i:word for word, i in word2id.items()}

Всего уникальных символов: 141526
Уникальных символов, втретившихся больше 2 раз: 35573


In [15]:
class TwitDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['words'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['sent'].values
        self.device = DEVICE


    def __len__(self):
        return self.length

    def __getitem__(self, index): 
        words = list(self.dataset[index])
        ids = torch.LongTensor([self.word2id[word] for word in words if word in self.word2id])
        y = [self.target[index]]
        return ids, y

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
        ids, y = list(zip(*batch))
        padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
        y = torch.Tensor(y).to(self.device)
        return padded_ids, y

In [16]:
data = twits[['words', 'sent']]
data

,words,sent
0,"[хоть, я, и, школотый, но, поверь, у, мы, то, ...",1
1,"[да, он, немного, похожий, на, он, но, мой, ма...",1
2,"[rt, katiacheh, ну, ты, идиотка, я, испугаться...",1
3,"[rt, кто, то, в, угол, сидеть, и, погибать, от...",1
4,"[вот, что, значит, страшилка, d, но, блин, пос...",1
...,...,...
226829,"[но, не, каждый, хотеть, что, то, исправлять, ...",0
226830,"[скучать, так, только, taaannyaaa, вправлять, ...",0
226831,"[вот, и, в, школа, в, говно, это, идти, уже, н...",0
226832,"[rt, lisaberoud, тауриэль, не, грусть]",0


In [17]:
train_data, val_data = train_test_split(data, test_size=0.2)

In [18]:
train_dataset = TwitDataset(train_data, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn,
                            sampler=train_sampler, batch_size=1024)

In [20]:
# train_data['sent'].values
batch = next(iter(train_iterator))
# batch[0].shape
# [id2word[int(i)] for i in batch[0][0]]

In [23]:
val_dataset = TwitDataset(train_data, word2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn,
                          sampler=val_sampler, batch_size=1024)

In [24]:
val_batch = next(iter(val_iterator))
val_batch[0].shape

torch.Size([1024, 28])

In [25]:
class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=180, out_features=1)
        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Sigmoid()

    def forward(self, word):
        #batch_size x seq_len
        embedded = self.embedding(word)
        #batch_size x seq_len x embedding_dim
        embedded = embedded.transpose(1,2)
        #batch_size x embedding_dim x seq_len
        feature_map_bigrams = self.dropout(self.pooling(self.relu(self.bigrams(embedded))))
        #batch_size x filter_count2 x seq_len* 
        feature_map_trigrams = self.dropout(self.pooling(self.relu(self.trigrams(embedded))))
        #batch_size x filter_count3 x seq_len*

        pooling1 = feature_map_bigrams.max(2)[0] 
        # batch_size x filter_count2
        pooling2 = feature_map_trigrams.max(2)[0]
        # batch_size x filter_count3
        concat = torch.cat((pooling1, pooling2), 1)
        # batch _size x (filter_count2 + filter_count3)
        logits = self.hidden(concat) 
        logits = self.out(logits)      
        return logits

In [26]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах

    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts)  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')      
    return  epoch_loss / len(iterator) # возвращаем среднее значение лосса по всей выборке

In [28]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            preds = model(texts)  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = f1(preds.round().long(), ys.long(), ignore_index=0)
            epoch_metric += batch_metric

            if not (i + 1) % int(len(iterator)/5):
              print(f'Val loss: {epoch_loss/i}, Val f1: {epoch_metric/i}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

In [29]:
model = CNN(len(word2id), 186)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCELoss()  

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [30]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(50):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)


starting Epoch 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


Train loss: 0.6716206915238324
Train loss: 0.6380948044251704
Train loss: 0.6219461084558413
Train loss: 0.6097646484272086
Train loss: 0.6017136364832691

Evaluating on train...
Val loss: 0.5767036378383636, Val f1: 0.6966602802276611
Val loss: 0.5665939173836639, Val f1: 0.6912545561790466
Val loss: 0.5638768609899741, Val f1: 0.6872933506965637
Val loss: 0.5630330096045844, Val f1: 0.6856608986854553
Val loss: 0.5623278001259113, Val f1: 0.6845225691795349

Evaluating on test...
Val loss: 0.5768646001815796, Val f1: 0.6960974931716919
Val loss: 0.5673409976821014, Val f1: 0.6875928640365601
Val loss: 0.565010093152523, Val f1: 0.6841810941696167
Val loss: 0.563693946214031, Val f1: 0.6829524636268616
Val loss: 0.5624010884898832, Val f1: 0.6839174032211304

starting Epoch 1
Training...
Train loss: 0.5705200977185193
Train loss: 0.5587630107782889
Train loss: 0.5536701094645721
Train loss: 0.5496592753225094
Train loss: 0.545320692761191

Evaluating on train...
Val loss: 0.5391111934

Хороший результат